# Statistical machine learning - Notebook 3, version for students
**Author: Michał Ciach**  
Exercises denoted with a star \* are optional. They may be more difficult or time-consuming.  


## Description
In today's class, we'll learn a method of statistical inference called *hypothesis testing*.   

*Theory time.* The general idea of statistical hypothesis testing is to set up a statistical hypothesis - e.g., that some parameter is greater than 0 - and try to *reject it* by showing that our data would be very unlikely to observe if this hypothesis was true.     

Why do we reject statistical hypotheses rather than confirm them? Because in general, confirming them is impossible using random samples. For example, let's suppose that we study the toxicity of a new drug by giving it to 10 rats and observing them for a week. Just because the rats didn't get sick may suggest that the drug is safe, but it doesn't prove it - after all, it's just 10 animals observed for a short time. On the other hand, if all the 10 animals get sick (and the control group is fine), we can definitely reject the hypothesis that the drug is safe.   

Another example: let's suppose we have a hypothesis that the average log-length of a human protein is equal 2.30, and we sample 200 proteins to verify it. If their average log-length turns out to be similar to 2.30 - for example, 2.31 with a standard deviation 0.1 - this doesn't prove our hypothesis. The true average log-length may still be equal, for example, 2.29 or 2.305. However, if the average log-length of our sample turns out to be 2.70 with a standard deviation 0.1, this is a strong evidence that our hypothesis is false.  

More generally: just because some data somewhat agree with our beliefs doesn't prove that our beliefs are true. However, if the data contradicts our beliefs, then it proves our beliefs are false.  





## Data & library imports

In [ ]:
!pip install gdown

In [1]:
!gdown https://drive.google.com/uc?id=1xOJfD-jexDbHSOCg1EiyAxqc5kXjMvX0
!gdown https://drive.google.com/uc?id=1y5NKR3aWB0DbAuSWcg6ffa1Atu2unpOA

Downloading...
From: https://drive.google.com/uc?id=1xOJfD-jexDbHSOCg1EiyAxqc5kXjMvX0
To: /content/protein_lengths.tsv
100% 29.3M/29.3M [00:00<00:00, 117MB/s]
Downloading...
From: https://drive.google.com/uc?id=1y5NKR3aWB0DbAuSWcg6ffa1Atu2unpOA
To: /content/citizen incomes.tsv
100% 40.6k/40.6k [00:00<00:00, 70.6MB/s]


In [2]:
# !pip install --upgrade scipy

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy.stats import t as tstud
from scipy.stats import ttest_ind, ttest_rel, ttest_1samp, norm, kstest, mannwhitneyu, shapiro, chisquare, chi2_contingency
import plotly.graph_objects as go
from statsmodels.stats.multitest import fdrcorrection

In [4]:
protein_lengths = pd.read_csv('protein_lengths.tsv', sep='\t')
protein_lengths['LogLength'] = np.log10(protein_lengths['Protein length'])
protein_lengths

,Scientific name,Common name,Protein ID,Protein length,LogLength
0,Homo sapiens,Human,NP_000005.3,1474,3.168497
1,Homo sapiens,Human,NP_000006.2,290,2.462398
2,Homo sapiens,Human,NP_000007.1,421,2.624282
3,Homo sapiens,Human,NP_000008.1,412,2.614897
4,Homo sapiens,Human,NP_000009.1,655,2.816241
...,...,...,...,...,...
648731,Imleria badia,Bay bolete (mushroom),KAF8560453.1,494,2.693727
648732,Imleria badia,Bay bolete (mushroom),KAF8560454.1,737,2.867467
648733,Imleria badia,Bay bolete (mushroom),KAF8560455.1,554,2.743510
648734,Imleria badia,Bay bolete (mushroom),KAF8560456.1,813,2.910091


In [5]:
human_protein_lengths = protein_lengths.loc[protein_lengths['Common name'] == 'Human'].copy()
# Note: without .copy(), some versions of Pandas may return a View.
# This may interfere with adding a new column to human_protein_lengths.
print(human_protein_lengths.head())
print()
print(human_protein_lengths.describe())

  Scientific name Common name   Protein ID  Protein length  LogLength
0    Homo sapiens       Human  NP_000005.3            1474   3.168497
1    Homo sapiens       Human  NP_000006.2             290   2.462398
2    Homo sapiens       Human  NP_000007.1             421   2.624282
3    Homo sapiens       Human  NP_000008.1             412   2.614897
4    Homo sapiens       Human  NP_000009.1             655   2.816241

       Protein length      LogLength
count   136193.000000  136193.000000
mean       692.655775       2.711540
std        746.993628       0.329892
min         12.000000       1.079181
25%        316.000000       2.499687
50%        514.000000       2.710963
75%        842.000000       2.925312
max      35991.000000       4.556194


In [6]:
citizen_incomes = pd.read_csv('citizen incomes.tsv', sep='\t')
citizen_incomes

,Country,Income
0,A,43806.926920
1,A,62825.327197
2,A,46660.685387
3,A,54625.879014
4,A,41976.428035
...,...,...
1995,B,55338.284370
1996,B,95699.796316
1997,B,34417.930506
1998,B,20637.868432


## Testing the value of the mean

To illustrate the basic concepts of statistical hypothesis testing, we'll start with simple tests for a hypothesis that the true mean value is equal to some value $\mu_0$, with an alternative hypothesis that it's different:  

$$H_0: \mu = \mu_0$$
$$H_1: \mu \neq \mu_0$$  
Note the lack of hats above any of the symbols - the hypotheses are about parameters, not estimators or any other random values.   

**Exercise 1.** Consider the human protein log-length data in the `human_protein_lengths` data frame. Consider the following two null hypotheses: $H_0^{(1)}: \mu = 2.711540$ and $H_0^{(2)}: \mu = 6$. We'll use the Student's t test to verify the hypotheses using a random sample.  

1. Select a random sample of protein log-lengths of size $N=20$.   
2. Calculate the test statistics for one-sample Student's t-tests with the assumption that the standard deviation is unknown and is estimated from the sample. Pay attention which kind of the variance estimator you need to use (biased or unbiased). How many test statistics do you need to calculate to test the two hypotheses, $H_0^{(1)}$ and $H_0^{(2)}$?   
3. Use the `tstud.ppf` (i.e. the quantile function of Student's t distribution) to calculate the critical set on the significance level 5% (i.e., Type I error risk 5%). Pay attention to the shape of the critical set - for our alternative hypothesis $H_1$, this set is a union of two semi-lines. How many quantile values do you need to calculate to test the two hypotheses, $H_0^{(1)}$ and $H_0^{(2)}$?   
  3.1. Based on the values of the test statistic and the critical set, do we reject our null hypotheses? Did we correctly detect which hypothesis is true and which is false?    
4. Use the `tstud.cdf` to calculate the p-values. Again, pay attention to the shape of the critical set. How many cdf values do you need to calculate to test the two hypotheses, $H_0^{(1)}$ and $H_0^{(2)}$?  
  4.1. Based on the p-values, do we reject any of our hypotheses on the significance level 5%? Did we correctly detect which hypothesis is true and which is false?     
5. Compare your results to the Student's t test implementation in `scipy`. The appropriate test has already been loaded in the *Data & imports* section.   
6. Are there any assumptions of the test that are violated? If so, how strongly and what effect could it have on the test result?  
7. Based on the results of this exercise, can you conclude that $N=20$ is enough to prove that $\mu=2.711540$? Does the answer to this question depend on $N$?   

**Exercise 2.** In this exercise, we'll see a more useful application of statistical hypothesis testing: comparing two populations. Say we want to use a random sample to check if the log-lengths of human proteins are, on average, higher than the ones of another organism - like the bay bolete (a kind of mushroom).

1. What are the appropriate null and alternative hypotheses if we want to use a random sample to show that human proteins are longer in terms of the average log-length?     
2. Select a random sample of human proteins ($N_\text{human}=20$) and a random sample of bay bolete proteins ($N_\text{bolete}=20$) from the `protein_lengths` data frame.
3. Use the two-sample Student's t-test implemented in the `ttest_ind` function from the `scipy` library to calculate the p-value. Pay attention to the `alternative` keyword, as well as to any keyword arguments that may correspond to assumptions such as the equality of variances.   
4. Based on the p-value, do we reject $H_0$ on a significance level 5%? Did we confirm that humans have longer or shorter proteins than the mushroom (in terms of the log-length)?      
5.\*\* Implement your own test and compare the p-value. You can use the equations described [here](https://en.wikipedia.org/wiki/Welch%27s_t-test).  
6. What happens if we take a reverse hypothesis - that humans have lower protein log-lengths than the mushroom? Did our results confirm anything now?  
  6.1. Is the result of the test true? Compare the true average log-lengths of the two organisms.  
  6.2. Is the result of the test from point 5 true?   
7.\* Do a test for a null hypothesis that the average protein log-lengths are equal, and an alternative that they are different (in any direction; a so-called *two-sided* alternative hypothesis). Explain the difference in the results compared to the previous points.    
  7.1. Roughly speaking, what is the difference in the shape of the critical region for a one-sided and a two-sided alternative hypothesis?   
  7.2.\* Calculate the ratio of the p-value for the two-sided alternative to the p-value for the one-sided alternative that the human log-lengths are smaller than the bolete log-lengths. Explain the result.  
8. Can we use the Student's t test to test protein lengths rather than log-lengths?   


## Non-parametric tests

The two-sample Student's t-test assumes a normal distribution of the populations. When this assumption is only slightly violated, like for the protein log-length data, the results may still be reliable, especially for large sample sizes. For many data sets, as the sample size increases, the distribution of the estimator of the mean converges to the Normal one (this is guaranteed by the Central Limit Theorem). This means that the estimator of the mean is distributed "more normally" than the original data (where "more normal" refers to the distance between the cumulative distribution functions). This increases the robustness of the t-test for small deviations from normality. However, when this assumption is strongly violated, like for the non-transformed protein length data, the results are no longer reliable. One way to solve this problem is to use non-parametric tests. A non-parametric test is defined as a test that does not rely on the assumption of a distribution of the data.   

One of the most common non-parametric tests is the Mann-Whitney U-test, also known as the two-sample Wilcoxon's test. It's often used as a replacement for the Student's t-test when the data is not distributed normally. However, the null hypotheses of these two tests are different, and it's important to understand this difference to avoid misleading results.

In contrast to the Student's t-test, the Mann-Whitney's one doesn't test the equality of parameters like the mean - hence the name *non-parametric*. Instead, its null hypothesis is that $\mathbb{P}(X > Y) = 1/2$, i.e., that if we take a random observation $X$ from the first sample, and a random observation $Y$ from the second sample, it's equally likely that the first is greater or smaller than the second. A one-sided alternative hypothesis may be, e.g., that  $\mathbb{P}(X > Y) > 1/2$, i.e., that samples from the first population tend to be larger than sample from the second one. In this case, we say that the first sample is *stochastically greater* than the second one.  

Sidenote: the actual null hypothesis of the Mann-Whitney test is slightly different, but the one described above is a very close approximation that's much simpler to interpret and use in practice.   



**Exercise 3.\*** Implement you own version of the Mann-Whitney's test. You can find the necessary equations [here](https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test) (use the normal approximation for the test statistic).

1. Use your implementation to test whether the protein lengths are higher in human than in the bay bolete (use a random sample of size $N$ of your choice).
2. Compare your results to the `mannwhitneyu` function from `scipy`. Pay attention to the default parameters to obtain identical results.  
3. Compare the results of the Mann-Whitney's test to the Student's t  test. Are the results of the two tests consistent? Can you conclude that one of the organisms has longer proteins? Is this a correct result?  


**Exercise 4.** The `citizen_income` data frame, loaded in the *Data & modules* section, contains the information about the yearly income in USD of randomly sampled individuals from two countries, encoded as Country `A` and Country `B`. Use an appropriate statistical test to check whether citizens of one of the countries earn more than citizens of the other country. If you use more than one test and get contradictory results, explain why that happens.   

**Exercise 5.\*\*** In this exercise, we'll see how the violations of test assumptions influence the distribution of the test statistics under the null hypothesis.   

1. Formulate a statistical hypothesis about the incomes of citizens of country `B` which you can test using any version of the Student's t test (regardless whether its assumptions are satisfied), and in which the null hypothesis $H_0$ is *true*.   
  1.1.\* Optionally, formulate a hypothesis which you can test using both Student's t and Mann-Whitney's u test.  
2. Which assumptions of your test are violated on this data set? Are they approximately satisfied for large sample sizes?    
3. Repeat the following $R=5000$ times:   
  3.1. Draw two random samples, each of size $N=10$.   
  3.2. Calculate the values of the Student's T statistic, either manually or using functions from `scipy`.   
  3.3.\* Calculate the Mann-Whitney's U statistic using functions from `scipy`.
  3.4. Save the values of the statistics in lists.  
4. Create a histogram that depicts the distribution of the statistic. Is the distribution correct (i.e. do they agree with the theoretical asumptions of the tests)? If not, how does it influence the test results?   
  4.1.\* Draw the probability density function of the theoretical distribution of the test statistics (under the null hypothesis) on the histograms.  
  4.2. Calculate the probability that your test makes a false positive error in this data set (i.e. that it incorrectly rejects a true null hypothesis; i.e. that the test statistic is within the theoretically calculated critical region).  
5. Did we analyze all the possible ways in which violated assumptions can influence test results? If not, what other problems or errors can be caused by the violated assumptions?   
6. What happens if you use protein log-lengths instead of citizen incomes?     



## Testing the distribution of the data

So far, we've covered testing the values of parameters and comparing two populations. Another important class are the *goodness-of-fit* tests, which are used to check if a sample can be assumed to come from some theoretical distribution, or to check if two variables have the same distributions. Probably the most commonly used tests are:  
* Chi-square test, designed for categorical data
* Kolmogorov-Smirnov test, designed for continuous data
* Shapiro-Wilk test, designed specifically for the Normal distribution

**Exercise 6.** Use the Shapiro-Wilk test (`scipy.shapiro`) to check the distribution of the test statistics from Exercise 5. Can we conclude that they are normally distributed? Does the answer depend on the value of $N$ and $R$ parameters?

**Exercise 7**. A bakery wants to introduce a new type of bread to its production line. It has developed three potential recipes and asked a sample of 120 people to pick their favorite one. Recipe 1 was picked by 30, Recipe 2 by 50 and Recipe 3 by 40 people. Does this mean that recipes will differ in popularity? Use a Chi-square test of goodness of fit (`scipy.chisquare`) to check this.  

**Exercise 8.\*\*** In this exercise, we'll use a goodness-of-fit test to crack the following message encoded with the [Ceasar cipher](https://en.wikipedia.org/wiki/Caesar_cipher):

*AOLJHLZHYJPWOLYPZVULVMAOLLHYSPLZARUVDUHUKZPTWSLZAJPWOLY
ZPAPZHAFWLVMZBIZAPABAPVUJPWOLYPUDOPJOLHJOSLAALYPUAOLWSH
PUALEAPZZOPMALKHJLYAHPUUBTILYVMWSHJLZKVDUAOLHSWOHILA*

Let $X_i$ be the frequency of the $i$-th letter of the alphabet in the above message. Use an appropriate goodness-of-fit test to find a shift of the alphabet for which the distribution of $X_i$ is statistically indistinguishable from the distribution of the letters in the English language. The frequency of each letter in the English language, and a function to shift the alphabet by a given number of letters, are given in the cell below.    




*Acknowledgement.* Exercise 8 was inspired by the following two resources: http://practicalcryptography.com/cryptanalysis/text-characterisation/chi-squared-statistic/ and https://ibmathsresources.com/2014/06/15/using-chi-squared-to-crack-codes/.

<center><img src='https://drive.google.com/uc?export=view&id=12CrUdXDAiltLBT26sG7HZ_HciIhvGyT8'></center>|